# Import

In [11]:
import numpy as np
import pandas as pd

# Data preporation

In [12]:
# Load data of previous day
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df = pd.read_csv(file_name)
rus_df.tail()

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
7493,2020-06-07,Челябинская область,Chelyabinsk region,74.0,115.0,2.0,70.0,3614.0,28.0,1721.0
7494,2020-06-07,Чеченская Республика,Chechen Republic,95.0,19.0,0.0,2.0,1345.0,14.0,990.0
7495,2020-06-07,Чукотский автономный округ,Chukotka Autonomous Okrug,87.0,4.0,0.0,2.0,106.0,1.0,65.0
7496,2020-06-07,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,73.0,0.0,53.0,2694.0,11.0,1429.0
7497,2020-06-07,Ярославская область,Yaroslavl region,76.0,70.0,0.0,7.0,3646.0,15.0,1176.0


In [13]:
# Read update data
# Result of data_parsing_SCRF.ipynb
file_name = f'covid19-russia-cases-upd-scrf.csv'
upd_df = pd.read_csv(file_name)
upd_df.sort_values('Confirmed', ascending=False).head()

,Region/City,Confirmed,Recovered,Deaths,Date,Day-Confirmed,Day-Deaths,Day-Recovered
29,Москва,197018,104347,2970,2020-06-08,2001.0,51.0,1633.0
30,Московская область,44983,12031,578,2020-06-08,751.0,11.0,569.0
63,Санкт-Петербург,18835,7915,392,2020-06-08,326.0,15.0,156.0
32,Нижегородская область,12442,4878,140,2020-06-08,305.0,4.0,41.0
66,Свердловская область,7668,3273,43,2020-06-08,249.0,2.0,34.0


In [14]:
# Strip text data
rus_df['Region/City'] = rus_df['Region/City'].astype('str').str.strip('\u200b') 
upd_df['Region/City'] = upd_df['Region/City'].astype('str').str.strip('\u200b')

# Date to datetime
upd_df['Date'] = pd.to_datetime(upd_df['Date'])
rus_df['Date'] = pd.to_datetime(rus_df['Date'])

# Updating

In [15]:
# Add region ID and Eng columns
regions_df = pd.read_csv('release/regions-info.csv')

right = regions_df[['Region_ID', 'Region', 'Region_eng']]
upd_df = upd_df.merge(right, left_on='Region/City', right_on='Region', how='left')
upd_df['Region/City-Eng'] = upd_df.Region_eng
upd_df = upd_df[['Date', 'Region/City', 'Region/City-Eng', 'Region_ID', 'Day-Confirmed', 'Day-Deaths', 'Day-Recovered', 'Confirmed', 'Deaths', 'Recovered']]

In [16]:
rus_df = pd.concat([rus_df, upd_df]).reset_index(drop=True)
rus_df.tail(10)

,Date,Region/City,Region/City-Eng,Region_ID,Day-Confirmed,Day-Deaths,Day-Recovered,Confirmed,Deaths,Recovered
7571,2020-06-08,Тульская область,Tula region,71.0,114.0,0.0,7.0,4734.0,49.0,2441.0
7572,2020-06-08,Тюменская область,Tyumen region,72.0,39.0,0.0,10.0,1947.0,12.0,1067.0
7573,2020-06-08,Удмуртская Республика,Udmurt republic,18.0,27.0,0.0,0.0,963.0,12.0,467.0
7574,2020-06-08,Ульяновская область,Ulyanovsk region,73.0,97.0,2.0,0.0,3265.0,16.0,954.0
7575,2020-06-08,Хабаровский край,Khabarovsk region,27.0,59.0,0.0,13.0,2596.0,17.0,1314.0
7576,2020-06-08,Ханты-Мансийский АО,Khanty-Mansiysk,86.0,129.0,1.0,60.0,3475.0,21.0,1186.0
7577,2020-06-08,Челябинская область,Chelyabinsk region,74.0,123.0,3.0,80.0,3737.0,31.0,1801.0
7578,2020-06-08,Чеченская Республика,Chechen Republic,95.0,20.0,1.0,3.0,1365.0,15.0,993.0
7579,2020-06-08,Ямало-Ненецкий АО,Yamalo-Nenets Autonomous Okrug,89.0,83.0,0.0,46.0,2777.0,11.0,1475.0
7580,2020-06-08,Ярославская область,Yaroslavl region,76.0,68.0,0.0,4.0,3714.0,15.0,1180.0


In [17]:
# Negative values fix
# Negative values can appear if some cases was removed from statistic
def negval_fix(reg):
    last = reg.iloc[-1]
    diff_c = last['Day-Confirmed']*(-1) if last['Day-Confirmed'] < 0 else 0
    diff_d = last['Day-Deaths']*(-1) if last['Day-Deaths'] < 0 else 0
    diff_r = last['Day-Recovered']*(-1) if last['Day-Recovered'] < 0 else 0
    
    last['Day-Confirmed'] += diff_c
    last['Day-Deaths']    += diff_d
    last['Day-Recovered'] += diff_r
    reg.iloc[-1] = last
    
    if diff_c + diff_d + diff_r > 0:
        print(diff_c, diff_d, diff_r, last['Region/City'])
        prev = reg.iloc[-2]
        prev['Confirmed'] -= diff_c
        prev['Deaths']    -= diff_d
        prev['Recovered'] -= diff_r
        reg.iloc[-2] = prev
    
    return reg
    
rus_df = rus_df.groupby('Region/City').apply(negval_fix).reset_index(drop=True)

In [18]:
# Manual fixes if needed
# rus_df.loc[287, 'Confirmed'] = 5
# rus_df.loc[321, 'Confirmed'] = 5
# rus_df.loc[359, 'Day-Confirmed'] = 0

# rus_df.loc[3775, 'Deaths'] = 7
# rus_df.loc[3775, 'Day-Deaths'] = 0

# Data checking

In [19]:
rus_sum = rus_df.loc[rus_df['Region/City'] != 'Diamond Princess']

print('Russia Confirmed:', rus_sum.groupby('Region/City')['Confirmed'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Confirmed'].sum().sum())
print('Russia Deaths:', rus_sum.groupby('Region/City')['Deaths'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Deaths'].sum().sum())
print('Russia Recovered:', rus_sum.groupby('Region/City')['Recovered'].max().sum(),
      '| By day:', rus_sum.groupby('Region/City')['Day-Recovered'].sum().sum())

rus_regs = rus_sum.groupby('Region/City')['Confirmed'].max().reset_index()
rus_regs['Confirmed-ByDay'] = rus_sum.groupby('Region/City')['Day-Confirmed'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Confirmed'] != rus_regs['Confirmed-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Recovered'].max().reset_index()
rus_regs['Recovered-ByDay'] = rus_sum.groupby('Region/City')['Day-Recovered'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Recovered'] != rus_regs['Recovered-ByDay']])

rus_regs = rus_sum.groupby('Region/City')['Deaths'].max().reset_index()
rus_regs['Deaths-ByDay'] = rus_sum.groupby('Region/City')['Day-Deaths'].sum().reset_index(drop=True)
print(rus_regs.loc[rus_regs['Deaths'] != rus_regs['Deaths-ByDay']])

# rus_regs.to_csv('rus_regs.csv')

Russia Confirmed: 476658.0 | By day: 476658.0
Russia Deaths: 5971.0 | By day: 5971.0
Russia Recovered: 230688.0 | By day: 230688.0
Empty DataFrame
Columns: [Region/City, Confirmed, Confirmed-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Recovered, Recovered-ByDay]
Index: []
Empty DataFrame
Columns: [Region/City, Deaths, Deaths-ByDay]
Index: []


# Data saving

In [20]:
file_name = 'release/covid19-russia-cases-scrf.csv'
rus_df.to_csv(file_name, index=False)